<a href="https://colab.research.google.com/github/Anna172/Mainor/blob/master/DiversEnd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import math
from tqdm import tqdm_notebook

In [0]:
#from tqdm import tqdm_notebook
#for i in tqdm_notebook(range(N)):
# просто цикл оборачиваешь в эту штуку и все

In [3]:
data = pd.read_csv("DomFull2.csv")
data.head()

,ID_W,ID_H,STATUS,A3,A3_3,NFM,B1_1,IDIND1,B1_2_1,B1_2_2,B1_4,B1_5,B1_8,B2_1,IDIND2,B2_2_1,B2_2_2,B2_4,B2_5,B2_8,B2_9_1,B3_1,IDIND3,B3_2_1,B3_2_2,B3_4,B3_5,B3_8,B3_9_1,B3_9_2,B4_1,IDIND4,B4_2_1,B4_2_2,B4_4,B4_5,B4_8,B4_9_1,B4_9_2,B4_9_3,...,FATHER,CHILDS,ID_I,MARST_B1,EDUC_B1,H4_B1,AGE_B1,INCOME_B1,MARST_B2,EDUC_B2,H4_B2,AGE_B2,INCOME_B2,MARST_B3,EDUC_B3,H4_B3,AGE_B3,INCOME_B3,MARST_B4,EDUC_B4,H4_B4,AGE_B4,INCOME_B4,MARST_M,EDUC_M,H4_M,AGE_M,J322_M,INCOME_M,MARST_F,EDUC_F,H4_F,AGE_F,J322_F,INCOME_F,ParentsFreeChilds,ParentsFreeChilds1,ParentsFreeChilds2,ParentsFreeChilds3,ParentsFreeChilds4
0,5,1001,3,1,NaN,2,1.0,1.0,NaN,NaN,2.0,1973.0,2.0,2.0,2.0,NaN,NaN,1.0,1971.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
1,5,1002,3,2,NaN,4,1.0,3.0,NaN,NaN,2.0,1955.0,3.0,2.0,4.0,NaN,NaN,1.0,1949.0,3.0,1.0,3.0,5.0,NaN,NaN,2.0,1984.0,NaN,4.0,4.0,4.0,6.0,NaN,NaN,1.0,1986.0,NaN,4.0,4.0,6.0,...,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
2,5,1003,3,3,NaN,1,1.0,7.0,NaN,NaN,2.0,1944.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
3,5,1004,3,4,NaN,1,1.0,8.0,NaN,NaN,2.0,1968.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
4,5,1005,3,5,NaN,2,1.0,9.0,NaN,NaN,2.0,1946.0,2.0,2.0,10.0,NaN,NaN,1.0,1951.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


In [4]:
data.shape

(131515, 377)

In [5]:
# все дети для которых будет предсказываться образование
childs = set()
for main_index in tqdm_notebook(range(len(data))):
  for i in range(1,5):
    Baby = "BABY" + str(i)
    if data.loc[main_index, Baby] != 0:
      childs.add(data.loc[main_index, Baby])
      
len(childs)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


4821

In [6]:
# дети, которые остались без отца/матери до 19 лет
childsFreeFather = set()
childsFreeMother = set()
for child in tqdm_notebook(childs):
  for i in range(1,5):
    Baby = "BABY" + str(i)
    if (len(data[(data[Baby] == child)]) == len(data[(data[Baby]==child) & (data['FATHER'] == 0)])) & (len(data[(data[Baby] == child)]) != 0):
      childsFreeFather.add(child)
    if (len(data[(data[Baby] == child)]) == len(data[(data[Baby]==child) & (data['MOTHER'] == 0)])) & (len(data[(data[Baby] == child)]) != 0):
      childsFreeMother.add(child)

      
len(childsFreeMother)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


142

In [7]:
len(childsFreeFather)

125

In [8]:
# Дети без мамы или папы
allfree = set()
allfree.update(childsFreeMother, childsFreeFather)
len(allfree)

267

In [9]:
# поиск детей, которые потеряли родителя до 19 и есть сведения об этом родителе
# таких не нашлось
newChildM = set()
for child in tqdm_notebook(childsFreeMother):
#for child in childsFreeFather:
  for ch in range(1,20):
    Baby = "IDIND" + str(ch)
    
    for index, row in data[data[Baby] == child].iterrows():
      number_members = int(row['NFM'])

      for i in range(2, number_members + 1):
        for j in range(i-1, 0, -1):
            #Ячейка  B@.9.@, при разных варианциях имеет разные форматы
            if (i >= 11 and i <= 16) and (j >= 10):
                cell = "B" + str(i) + "9_" + str(j)
            elif (i >= 17 and i <= 24) and (j >= 10):
                cell = "B" + str(i) + "_9" + str(j)
            else:
                cell = "B" + str(i) + "_9_" + str(j)
            
            #Если член семьи является отцом/матерью
            if (j == ch) & (row[cell] == 3):
                gender = "B" + str(i) + "_4"
                status = "B" + str(i) + "_2_2"
                if (row[gender] == 1) & ((row[status] == 2) ^ (row[status] == 3)):
                  vsp = []
                  vsp.append(child)
                  bd = "B" + str(j) + "5"
                  lwf = row["YEAR"] - row[bd]
                  vsp.append(lwf)
                  vsp = tuple(vsp)
                  newChildM.add(vsp)

             #Если член семьи является сыном/дочерью       
            elif (i == ch) & (row[cell] == 5):
              gender = "B" + str(j) + "_4"
              status = "B" + str(j) + "_2_2"
              if (row[gender] == 1) & ((row[status] == 2) ^ (row[status] == 3)):
                vsp = []
                vsp.append(child)
                bd = "B" + str(i) + "5"
                lwf = row["YEAR"] - row[bd]
                vsp.append(lwf)
                vsp = tuple(vsp)
                newChildM.add(vsp)
len(newChildM)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0

In [10]:
# Тогда найдем тех детей без родичей, у которых были отчим/мачеха 
newChildF = set() # это будет пара ребенок отчим
for child in tqdm_notebook(allfree):
#for child in childsFreeFather:
  for ch in range(1,20):
    Baby = "IDIND" + str(ch)
    
    for index, row in data[data[Baby] == child].iterrows():
      number_members = int(row['NFM'])

      for i in range(2, number_members + 1):
        for j in range(i-1, 0, -1):
            #Ячейка  B@.9.@, при разных варианциях имеет разные форматы
            if (i >= 11 and i <= 16) and (j >= 10):
                cell = "B" + str(i) + "9_" + str(j)
            elif (i >= 17 and i <= 24) and (j >= 10):
                cell = "B" + str(i) + "_9" + str(j)
            else:
                cell = "B" + str(i) + "_9_" + str(j)
            
            bd = "B" + str(j) + "_5"
            #Если член семьи является отцом/матерью
            if (j == ch) & (row[cell] == 3) & (row['YEAR'] - row[bd] < 19):
              vsp = []
              vsp.append(child)
              IDIND = "IDIND" + str(i)
              vsp.append(row[IDIND])
              vsp = tuple(vsp)
              newChildF.add(vsp)
            #Если член семьи является сыном/дочерью 
            bd = "B" + str(i) + "_5" 
            if (i == ch) & (row[cell] == 5) & (row['YEAR'] - row[bd] < 19):
              vsp = []
              vsp.append(child)
              IDIND = "IDIND" + str(j)
              vsp.append(row[IDIND])
              vsp = tuple(vsp)
              newChildF.add(vsp)


len(newChildF)       


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


23

In [11]:
print(newChildF)

{(8678, 8676.0), (8677, 8676.0), (53031, 33211.0), (21718, 3176.0), (39524, 39522.0), (24865, 23840.0), (8648, 8645.0), (40464, 40465.0), (31241, 24909.0), (36554, 36552.0), (22202, 23540.0), (1576, 11690.0), (16722, 16720.0), (37495, 37493.0), (4385, 12427.0), (19229, 19226.0), (36086, 36085.0), (17989, 17988.0), (22315, 22313.0), (38720, 42916.0), (14733, 2537.0), (27012, 27014.0), (16204, 11005.0)}


In [14]:
# для этих пар ребеноk отчиm/м найдем найдем кол-во лет существования этой пары
ycho = [] # пары ребенок + год рождения + отчим/м + начальный год + конечный год
ych = [] # пара ребенок + год его появления в данных

for child in tqdm_notebook(newChildF):
  ycho.append([child[0], 0, child[1], 1000000, 0, 0])
  ych.append([child[0], 1000000])
  for ch in range(1,20):
    IDINDB = "IDIND" + str(ch)
    bd = "B" + str(ch) + "_5"

    for index, row in data[(data[IDINDB] == child[0])].iterrows():
        for i in range(len(ych)):
          if (ych[i][0] == child[0]) & (ych[i][1] > row['YEAR']):
            ych[i][1] = row['YEAR']

    for o in range(1,20):
      IDINDO = 'IDIND' + str(o)
      gender = "B" + str(o) + "_4"
    
      for index, row in data[(data[IDINDB] == child[0]) & (data[IDINDO] == child[1])].iterrows():
        for i in range(len(ycho)):

          if (ycho[i][0] == child[0]) & (ycho[i][2] == child[1]) & (ycho[i][5] == 0) & (not math.isnan(row[gender])):
            ycho[i][5] = row[gender]
          
          if (ycho[i][0] == child[0]) & (ycho[i][1] == 0) & (not math.isnan(row[bd])):
            ycho[i][1] = row[bd]

          if (ycho[i][0] == child[0]) & (ycho[i][2] == child[1]):
            if ycho[i][3] > row['YEAR']:
              ycho[i][3] = row['YEAR']

            if (ycho[i][4] < row['YEAR']):
              if (ycho[i][1] != 0) & (ycho[i][1] + 18 < row['YEAR']):
                ycho[i][4] = ycho[i][1] + 18
              elif (ycho[i][1] != 0) & (ycho[i][1] + 18 > row['YEAR']):
                ycho[i][4] = row['YEAR']
              else:
                ycho[i][4] = row['YEAR']
                    
            

        
len(ycho)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


23

In [15]:
# для этих пар ребеноk отчиm/м найдем найдем кол-во лет существования этой пары
for i in range(len(ycho)):
  ycho[i].append(ycho[i][4] - ycho[i][3] + 1)

ycho


[[8678, 1989.0, 8676.0, 1994.0, 2007.0, 2.0, 14.0],
 [8677, 1986.0, 8676.0, 1994.0, 2004.0, 2.0, 11.0],
 [53031, 1998.0, 33211.0, 2016.0, 2016.0, 2.0, 1.0],
 [21718, 1985.0, 3176.0, 2003.0, 2003.0, 2.0, 1.0],
 [39524, 1993.0, 39522.0, 2010.0, 2011.0, 2.0, 2.0],
 [24865, 1991.0, 23840.0, 2005.0, 2009.0, 2.0, 5.0],
 [8648, 1976.0, 8645.0, 1994.0, 1994.0, 2.0, 1.0],
 [40464, 1992.0, 40465.0, 2010.0, 2010.0, 1.0, 1.0],
 [31241, 1991.0, 24909.0, 2008.0, 2009.0, 2.0, 2.0],
 [36554, 1997.0, 36552.0, 2010.0, 2015.0, 2.0, 6.0],
 [22202, 1995.0, 23540.0, 2004.0, 2013.0, 2.0, 10.0],
 [1576, 1980.0, 11690.0, 1996.0, 1998.0, 2.0, 3.0],
 [16722, 1988.0, 16720.0, 2000.0, 2006.0, 2.0, 7.0],
 [37495, 1992.0, 37493.0, 2010.0, 2010.0, 2.0, 1.0],
 [4385, 1986.0, 12427.0, 1996.0, 2004.0, 2.0, 9.0],
 [19229, 1984.0, 19226.0, 2001.0, 2002.0, 2.0, 2.0],
 [36086, 1994.0, 36085.0, 2010.0, 2012.0, 2.0, 3.0],
 [17989, 1987.0, 17988.0, 2001.0, 2005.0, 2.0, 5.0],
 [22315, 1990.0, 22313.0, 2003.0, 2008.0, 2.0, 6.0],

In [16]:
ych

[[8678, 1994.0],
 [8677, 1994.0],
 [53031, 2016.0],
 [21718, 2003.0],
 [39524, 2010.0],
 [24865, 2005.0],
 [8648, 1994.0],
 [40464, 2010.0],
 [31241, 2008.0],
 [36554, 2010.0],
 [22202, 2003.0],
 [1576, 1994.0],
 [16722, 2000.0],
 [37495, 2010.0],
 [4385, 1994.0],
 [19229, 2001.0],
 [36086, 2010.0],
 [17989, 2001.0],
 [22315, 2003.0],
 [38720, 2010.0],
 [14733, 1998.0],
 [27012, 2006.0],
 [16204, 1998.0]]

In [17]:
# просто проверка: у 20 из 23 отчимы с начала исчесления, так что точноее время их с ребеноком не известно
a = []
for i in range(len(ych)):
  for j in range(len(ycho)):
    if (ycho[i][3] != ych[i][1]) & (ycho[i][0] == ych[j][0]):
      a.append(ych[i][0])
      print(ych[i][0], ycho[i][2], ycho[i][5])
len(a)

22202 23540.0 2.0
1576 11690.0 2.0
4385 12427.0 2.0


3

In [243]:
data.loc[data["IDIND5"] == 12427, 'YEAR']


9315     1996
13113    1998
17056    2000
21085    2001
25666    2002
30363    2003
35097    2004
39823    2005
44805    2006
50308    2007
55669    2008
61005    2009
Name: YEAR, dtype: int64

In [18]:
vsp = set()
for i in range(len(ycho)):
  vsp.add(ycho[i][0])

len(vsp)

23

In [0]:
data = data.assign(ParentsFreeChilds1=np.zeros(data.shape[0]))
data = data.assign(ParentsFreeChilds2=np.zeros(data.shape[0]))
data = data.assign(ParentsFreeChilds3=np.zeros(data.shape[0]))
data = data.assign(ParentsFreeChilds4=np.zeros(data.shape[0]))

In [19]:
# Осталось записать полученные данные в данные: -1, если без отца и отчима, > 0, если с отчимом, это кол-во лет с отчимом))

for main_index in tqdm_notebook(range(len(data))):
  for ch in range(1, 5):
    Baby = 'BABY' + str(ch)
    ParentsFreeChilds = 'ParentsFreeChilds' + str(ch)
    
    if data.loc[main_index, Baby] in allfree:
      data.loc[main_index, ParentsFreeChilds] = -1

    if data.loc[main_index, Baby] in vsp:
      for i in range(len(ycho)):
        if ycho[i][0] == data.loc[main_index, Baby]:
          data.loc[main_index, ParentsFreeChilds] = ycho[i][6]
          
          if ycho[i][5] == 1:
            data.loc[main_index, 'FATHER'] = -ycho[i][2]

          elif ycho[i][5] == 2:
            data.loc[main_index, 'MOTHER'] = -ycho[i][2]











/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [29]:
UNDdata = pd.read_csv("Ind.csv")
UNDdata.head()

,ID_W,ID_H,EDUC,H4,AGE,INCOME,IDIND
0,5,1001,18.0,1,21,160000.0,-1
1,5,1001,16.0,2,23,600000.0,-2
2,5,1002,16.0,1,39,1200000.0,-3
3,5,1002,14.0,2,45,39000.0,-4
4,5,1002,NaN,3,10,NaN,-5


In [31]:
UNDdata.shape

(353827, 7)

In [32]:
# добавим индивидуальные данные о отчимах/маихах
# Осталось записать полученные данные в данные: -1, если без отца и отчима, > 0, если с отчимом, это кол-во лет с отчимом))

for main_index in tqdm_notebook(range(len(data))):
  for ch in range(1, 5):
    Baby = 'BABY' + str(ch)
    
    if data.loc[main_index, Baby] in vsp:
      for index, row in UNDdata[(UNDdata['IDIND'] == data.loc[main_index, 'MOTHER']) & (UNDdata['ID_W'] == data.loc[main_index, 'ID_W']) & (UNDdata['ID_H'] == data.loc[main_index, 'ID_H'])].iterrows():
        data.loc[main_index, 'EDUC_M'] = row['EDUC']
        data.loc[main_index, 'H4_M'] = row['H4']
        data.loc[main_index, 'AGE_M'] = row['AGE']
        data.loc[main_index, 'INCOME_M'] = row['INCOME']

      for index, row in UNDdata[(UNDdata['IDIND'] == data.loc[main_index, 'FATHER']) & (UNDdata['ID_W'] == data.loc[main_index, 'ID_W']) & (UNDdata['ID_H'] == data.loc[main_index, 'ID_H'])].iterrows():
        data.loc[main_index, 'EDUC_F'] = row['EDUC']
        data.loc[main_index, 'H4_F'] = row['H4']
        data.loc[main_index, 'AGE_F'] = row['AGE']
        data.loc[main_index, 'INCOME_F'] = row['INCOME']







/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [34]:
len(data.loc[(data["ParentsFreeChilds1"] != 0) & (data["MOTHER"] < 0) & (data["EDUC_M"] != 0), "EDUC_M"])

43

In [0]:
data.to_csv("TheEnd.csv", index=False)